In [119]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import itertools
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [3]:
df = pd.read_csv('df_df+log.csv')

In [4]:
df.head();

In [5]:
df.shape

(8334, 30)

In [6]:
X_c = df[['log_time_allowed', 'log_goal']]

In [7]:
X_c.shape

(8334, 2)

In [8]:
# from sklearn.preprocessing import StandardScaler
# X_c_s = pd.DataFrame(data = StandardScaler().fit_transform(X_c), columns = ['log_time_allowed', 'log_goal']).head()

In [9]:
X_d = pd.get_dummies(df[['cat_slug', 'staff_pick', 'country', 'loc_state', 'cat_name']], drop_first=True, prefix_sep='_')

In [10]:
X = pd.concat([X_c,X_d],1)
X;

In [11]:
X.dropna(0,inplace=True)

In [12]:
X.shape

(8334, 675)

In [13]:
y = pd.get_dummies(df[['state']], drop_first=True)
y;

In [103]:
pca = PCA(n_components=150)
principalComponents = pca.fit_transform(X)

In [104]:
pca.components_;

In [105]:
pca.mean_;

In [107]:
pca.explained_variance_ratio_;

In [108]:
round(sum(list(pca.explained_variance_ratio_)*100),1)

95.3

In [113]:
cols = []
counter = 1
while counter-1 < (principalComponents.shape)[1]:
    counter_str = str(counter)
    cols.append('PC'+counter_str)
    counter+=1   

In [115]:
cols[0],cols[-1]

('PC1', 'PC150')

In [116]:
X_1 = pd.DataFrame(data = principalComponents, columns = cols)

In [117]:
y_1 = y

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size = 0.3, shuffle=True)

In [123]:
def summary(y_test,y_hat_te,y_train,y_hat_tr):
    print('Accuracy Score- Training Model: '+str(round(accuracy_score(y_train, y_hat_tr)*100,1))+'%. Model on testing data: '+str(round(accuracy_score(y_test, y_hat_te)*100,1))+'%.')
    print('Recall Score- Training Model: '+str(round(recall_score(y_train, y_hat_tr)*100,1))+'%. Model on testing data: '+str(round(recall_score(y_test, y_hat_te)*100,1))+'%.')
    print('Precision score - Training Model: '+str(round(precision_score(y_train, y_hat_tr)*100,1))+'%. Model on testing data: '+str(round(precision_score(y_test, y_hat_te)*100,1))+'%.')
    print('F1 score - Training Model: '+str(round(f1_score(y_train, y_hat_tr)*100,1))+'%. Model on testing data: '+str(round(f1_score(y_test, y_hat_te)*100,1))+'%.')

### Random Forest

In [120]:
forest_vanilla = RandomForestClassifier()
forest_vanilla.fit(X_train, y_train)

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [126]:
y_hat_train_v_rf = forest_vanilla.predict(X_train)
y_hat_test_v_rf = forest_vanilla.predict(X_test)
confusion_matrix(y_train, y_hat_train_v_rf)
confusion_matrix(y_test, y_hat_test_v_rf)

array([[ 626,  314],
       [ 388, 1173]])

In [130]:
y_score_rf_v = logreg_vanilla.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_score_rf_v)
print('AUC: {}'.format(auc(fpr, tpr)))

AUC: 0.866065124647321


In [131]:
summary(y_test, y_hat_test_v_rf, y_train, y_hat_train_v_rf)

Accuracy Score- Training Model: 98.6%. Model on testing data: 71.9%.
Recall Score- Training Model: 98.1%. Model on testing data: 75.1%.
Precision score - Training Model: 99.6%. Model on testing data: 78.9%.
F1 score - Training Model: 98.8%. Model on testing data: 77.0%.


In [150]:
rf = RandomForestClassifier(criterion='entropy')

In [153]:
param_grid_rf = {
    "max_depth": list(range(2,80,2)),
    'min_samples_split': list(np.arange(0.1,1.0,0.1))
}

In [154]:
grid_clf = GridSearchCV(rf, param_grid_rf, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

y_hat_train = grid_clf.predict(X_train)
y_hat_test = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, y_hat_train)
val_accuracy = accuracy_score(y_test, y_hat_test)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.2

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimato

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

...

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

Grid Search found the following optimal parameters: 
max_depth: 26
min_samples_split: 0.1

Training Accuracy: 75.36%
Validation accuracy: 72.93%


In [ ]:
# Grid Search found the following optimal parameters: 
# max_depth: 26
# min_samples_split: 0.1

# Training Accuracy: 75.36%
# Validation accuracy: 72.93%

In [155]:
summary(y_test, y_hat_test, y_train, y_hat_train)

Accuracy Score- Training Model: 75.4%. Model on testing data: 72.9%.
Recall Score- Training Model: 90.7%. Model on testing data: 88.6%.
Precision score - Training Model: 74.9%. Model on testing data: 73.5%.
F1 score - Training Model: 82.0%. Model on testing data: 80.3%.


### Logistic Regression

In [135]:
logreg = LogisticRegression(penalty='l1', solver='liblinear')
model_log = logreg.fit(X_train, y_train)

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [136]:
y_hat_test_lr_ = logreg.predict(X_test)
y_hat_train_lr_ = logreg.predict(X_train)
confusion_matrix(y_train, y_hat_train_lr_)
confusion_matrix(y_test, y_hat_test_lr_)

array([[ 675,  265],
       [ 293, 1268]])

In [139]:
y_score_lr_ = logreg.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_score_lr_)
print('AUC: {}'.format(auc(fpr, tpr)))

AUC: 0.8661148745348726


In [141]:
summary(y_test, y_hat_test_lr_, y_train, y_hat_train_lr_)

Accuracy Score- Training Model: 80.4%. Model on testing data: 77.7%.
Recall Score- Training Model: 84.5%. Model on testing data: 81.2%.
Precision score - Training Model: 83.9%. Model on testing data: 82.7%.
F1 score - Training Model: 84.2%. Model on testing data: 82.0%.


### XGBoost

In [156]:
XGBoost = xgb.XGBClassifier()
XGBoost.fit(X_train, y_train)
y_hat_train_XG_v = XGBoost.predict(X_train)
y_hat_test_XG_v = XGBoost.predict(X_test)
training_accuracy = accuracy_score(y_train, y_hat_train_XG_v)
val_accuracy = accuracy_score(y_test, y_hat_test_XG_v)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training Accuracy: 82.27%
Validation accuracy: 76.09%


In [157]:
summary(y_test, y_hat_test_XG_v, y_train, y_hat_train_XG_v)

Accuracy Score- Training Model: 82.3%. Model on testing data: 76.1%.
Recall Score- Training Model: 88.1%. Model on testing data: 83.0%.
Precision score - Training Model: 84.1%. Model on testing data: 79.6%.
F1 score - Training Model: 86.0%. Model on testing data: 81.3%.


In [165]:
param_grid_XG = {
    "learning_rate": [0.1,0.2],
    'max_depth': [20,30,60],
    'min_child_weight': [10,20],
    'subsample': [0.5,0.7],
}

In [166]:
grid_clf = GridSearchCV(XGBoost, param_grid_XG, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

y_hat_train_XG = grid_clf.predict(X_train)
y_hat_test_XG = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, y_hat_train_XG)
val_accuracy = accuracy_score(y_test, y_hat_test_XG)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d ar

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was pass

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was pass

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was pass

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was pass

Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 20
min_child_weight: 20
subsample: 0.7

Training Accuracy: 92.71%
Validation accuracy: 76.45%


In [ ]:
# Grid Search found the following optimal parameters: 
# learning_rate: 0.1
# max_depth: 20
# min_child_weight: 20
# subsample: 0.7

# Training Accuracy: 92.71%
# Validation accuracy: 76.45%

In [167]:
summary(y_test, y_hat_test_XG, y_train, y_hat_train_XG)

Accuracy Score- Training Model: 92.7%. Model on testing data: 76.4%.
Recall Score- Training Model: 95.0%. Model on testing data: 81.9%.
Precision score - Training Model: 93.4%. Model on testing data: 80.6%.
F1 score - Training Model: 94.2%. Model on testing data: 81.3%.


### SVM

In [174]:
from sklearn import svm

clf = svm.SVC(kernel='poly', degree=4)
clf.fit(X_train, y_train)

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=4, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [175]:
clf.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [176]:
accuracy_score(y_test,clf.predict(X_test))

0.6241503398640543